<a href="https://colab.research.google.com/github/Akashthakur0326/ResumeRecommender-MLops/blob/main/rr_categorizer_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dagshub mlflow

In [ ]:
MLFLOW_TRACKING_URI= "https://dagshub.com/Akashthakur0326/ResumeRecommender-MLops.mlflow"
MLFLOW_TRACKING_USERNAME= "Akashthakur0326"
MLFLOW_TRACKING_PASSWORD="d2a8af2664d554a5366508dd80f33e027eb377c2"

In [ ]:
import os
import mlflow

# Configuration
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD
tracking_uri = MLFLOW_TRACKING_URI

mlflow.set_tracking_uri(tracking_uri)

"""mlflow.set_experiment("colab-experiment-setup")
with mlflow.start_run():
    mlflow.log_param("lr", 0.01)"""

In [ ]:
import pandas as pd

df1= pd.read_csv("/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/labeled_jobs.csv")
df2= pd.read_csv("/content/drive/Othercomputers/My laptop/Desktop/ResumeRecommenderMLops/data/synthetic.csv")


In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df.info()

In [ ]:
df.sample(5)

In [ ]:
col = "llm_based_category"

l = df[col].unique().tolist()

In [ ]:
df[col].value_counts().to_dict()


In [ ]:
vc = df[col].value_counts()
vc[vc <= 10].sum()

In [ ]:
import pandas as pd

drop_categories = {
    "Python Developer",
    "MLOps Engineer",
    "QA Engineer",
    "Applied Machine Learning Engineer",
    "Applied ML Engineer",
    "Platform Engineer",
    "Deep Learning Engineer",
    "normalized_job_category",
    "React Developer",
    "Database Engineer",
    "Research Scientist",
    "Site Reliability Engineer",
    "Analytics Engineer",
    "Mobile Developer",
    "FinOps Engineer",
}

merge_map = {
    "AR/VR Engineer": "Graphics Programmer",
    "iOS Developer": "Mobile Application Developer",
    "Firmware Engineer": "Embedded Systems Engineer",
    "NoSQL Developer": "Data Engineer",
    "Quality Assurance Engineer": "SDET",
    "Quality Assurance Analyst": "SDET",
    "Test Automation Engineer": "Automation Test Engineer",
    "Information Security Engineer": "Security Engineer",
    "Cybersecurity Analyst": "Cybersecurity Engineer",
    "Network Security Engineer": "Security Engineer",
    "Cloud Security Engineer": "Security Engineer",
    "Application Security Engineer": "Security Engineer",
    "Backend Developer": "Backend Engineer",
    "Software Architect": "Software Engineer",
    "Technical Lead": "Software Engineer",
    "Engineering Manager": "Software Engineer",
    "Program Manager": "Technical Product Manager",
    "Project Manager": "Technical Product Manager",
    "Product Manager": "Technical Product Manager",
    "Product Designer": "Product Engineer",
    "UI/UX Designer": "Product Engineer",
    "UX Researcher": "Product Engineer",
}

df["llm_based_category"] = df["llm_based_category"].replace(merge_map)

df = df[~df["llm_based_category"].isin(drop_categories)].reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
df[col].value_counts().to_dict()

Architecture selection

In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

mlflow.set_experiment("job-categorizer-phase1-baselines")

In [ ]:
df_clean = df.dropna(subset=["description", "llm_based_category"])
X = df_clean["description"].astype(str)
y = df_clean["llm_based_category"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
print(f"Training on {len(X_train)} samples, Validating on {len(X_val)} samples.")

#Baseline 1 — Logistic Regression
with mlflow.start_run(run_name="tfidf_logreg_baseline"):
    pipeline_lr = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1, 2), stop_words="english")),
        ("clf", LogisticRegression(max_iter=1000, n_jobs=-1, class_weight="balanced", solver="saga"))
    ])

    pipeline_lr.fit(X_train, y_train)
    preds = pipeline_lr.predict(X_val)

    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    mlflow.log_params({"model": "logistic_regression", "ngram": "(1,2)"})
    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    # Save the model artifact
    mlflow.sklearn.log_model(pipeline_lr, "model")

    print(f"\nLogistic Regression -> Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

In [ ]:
with mlflow.start_run(run_name="tfidf_linear_svm_baseline"):
    pipeline_svm = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=50000, ngram_range=(1, 2), stop_words="english")),
        ("clf", LinearSVC(class_weight="balanced", dual="auto")) # dual='auto' suppresses newer sklearn warnings
    ])

    pipeline_svm.fit(X_train, y_train)
    preds = pipeline_svm.predict(X_val)

    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    mlflow.log_params({"model": "linear_svm", "ngram": "(1,2)"})
    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    # Save the model artifact
    mlflow.sklearn.log_model(pipeline_svm, "model")

    print(f"Linear SVM          -> Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

mlflow.set_experiment("job-categorizer-phase1-refined")

with mlflow.start_run(run_name="tfidf_svm_refined_v1"):

    pipeline_svm = Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=5000,      # CHANGED: 50k -> 5k to prevent overfitting
            min_df=5,               # NEW: Ignore words appearing in < 5 docs
            ngram_range=(1, 2),     # Keep bigrams (e.g., "machine learning")
            stop_words="english",
            sublinear_tf=True       # NEW: Log scaling for term frequency
        )),
        ("clf", LinearSVC(
            class_weight="balanced",
            C=0.5,                  # NEW: Slightly stronger regularization (default is 1.0)
            dual="auto",
            random_state=42
        ))
    ])

    # Train
    pipeline_svm.fit(X_train, y_train)

    # Predict
    preds = pipeline_svm.predict(X_val)

    # Metrics
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    # Logging
    mlflow.log_params({
        "model": "linear_svm",
        "max_features": 5000,
        "min_df": 5,
        "sublinear_tf": True,
        "C": 0.5
    })
    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    # Save Model
    mlflow.sklearn.log_model(pipeline_svm, "model")

    print(f"Refined SVM Results:")
    print(f"Accuracy: {acc:.4f} (Previous: 0.7331)")
    print(f"Macro F1: {f1:.4f} (Previous: 0.6890)")

In [ ]:
from mlflow.models import infer_signature

# 1. Define everything in one place
RUN_CONFIG = {
    "model_type": "linear_svm",
    "ngram_range": (1, 2),
    "max_features": 5000,
    "min_df": 5,
    "sublinear_tf": True,
    "C": 0.5,
    "text_source": "description_only"
}

with mlflow.start_run(run_name="svm_config_driven_refined"):
    # 2. Log the entire config dictionary
    mlflow.log_params(RUN_CONFIG)

    # 3. Use the config to build the pipeline (Corrected variable name)
    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=RUN_CONFIG["max_features"],
            min_df=RUN_CONFIG["min_df"],
            ngram_range=RUN_CONFIG["ngram_range"],
            sublinear_tf=RUN_CONFIG["sublinear_tf"],
            stop_words="english"
        )),
        ("clf", LinearSVC(
            C=RUN_CONFIG["C"],
            class_weight="balanced",
            dual="auto",
            random_state=42
        ))
    ])

    # 4. Train using the NEW pipeline
    pipeline.fit(X_train, y_train)

    # 5. Predict and Evaluate
    preds = pipeline.predict(X_val)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    # 6. Log Metrics
    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    # 7. Infer Signature (Captures data schema for the Model Registry)
    signature = infer_signature(X_val, preds)

    # 8. Save Model with Signature
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="model",
        signature=signature
    )

    print(f"Refined SVM Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Macro F1: {f1:.4f}")

In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from mlflow.models import infer_signature

df['title_clean'] = df['title'].fillna('')
df['desc_clean'] = df['description'].fillna('')

df['combined_text'] = df['title_clean'] + " " + df['desc_clean']

X_combined = df['combined_text'].astype(str)
y = df['llm_based_category']

X_train, X_val, y_train, y_val = train_test_split(
    X_combined, y, test_size=0.2, stratify=y, random_state=42
)

RUN_CONFIG = {
    "model_type": "linear_svm",
    "text_source": "title_plus_description",
    "ngram_range": (1, 2),
    "max_features": 7000,
    "min_df": 5,
    "sublinear_tf": True,
    "C": 0.5
}

with mlflow.start_run(run_name="svm_title_desc_combined"):
    mlflow.log_params(RUN_CONFIG)

    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=RUN_CONFIG["max_features"],
            min_df=RUN_CONFIG["min_df"],
            ngram_range=RUN_CONFIG["ngram_range"],
            sublinear_tf=RUN_CONFIG["sublinear_tf"],
            stop_words="english"
        )),
        ("clf", LinearSVC(
            C=RUN_CONFIG["C"],
            class_weight="balanced",
            dual="auto",
            random_state=42
        ))
    ])

    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)

    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    signature = infer_signature(X_val, preds)
    mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="job_categorizer_model",
        signature=signature
    )

    print(f"🚀 Title+Description Fusion Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Macro F1: {f1:.4f}")

HYPERPARAMETER TUNING

In [ ]:
!pip install optuna

In [ ]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from mlflow.models import infer_signature

def objective(trial):
    """
    Optimizes Linear SVM on Title+Description data using Stratified CV.
    """
    params = {
        # Vectorizer Params
        "tfidf__max_features": trial.suggest_int("max_features", 5000, 30000, step=1500), # Exploring higher range
        "tfidf__min_df": trial.suggest_int("min_df", 2, 10),
        "tfidf__ngram_range": trial.suggest_categorical("ngram_range", [(1, 1), (1, 2), (1, 3)]), # Added trigrams
        "tfidf__sublinear_tf": trial.suggest_categorical("sublinear_tf", [True, False]),

        # SVM Params
        "clf__C": trial.suggest_float("C", 0.1, 10.0, log=True),
        "clf__loss": trial.suggest_categorical("loss", ["hinge", "squared_hinge"]),
    }

    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=params["tfidf__max_features"],
            min_df=params["tfidf__min_df"],
            ngram_range=params["tfidf__ngram_range"],
            sublinear_tf=params["tfidf__sublinear_tf"],
            stop_words="english"
        )),
        ("clf", LinearSVC(
            C=params["clf__C"],
            loss=params["clf__loss"],
            class_weight="balanced",
            dual=True,
            random_state=42
        ))
    ])

    cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    scores = cross_val_score(
        pipeline, X_train, y_train,
        cv=cv_strategy,
        scoring="f1_macro",
        n_jobs=-1
    )

    return scores.mean()

mlflow.set_experiment("job-categorizer-optuna-tuning")

print("🧠 Starting Optuna Optimization for Title+Desc Fusion...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("\n🎉 Optimization Complete!")
print(f"🏆 Best CV Macro F1: {study.best_value:.4f}")
print(f"⚙️ Best Params: {study.best_params}")

best_params = study.best_params

with mlflow.start_run(run_name="optuna_champion_title_desc"):

    final_pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            max_features=best_params["max_features"],
            min_df=best_params["min_df"],
            ngram_range=best_params["ngram_range"],
            sublinear_tf=best_params["sublinear_tf"],
            stop_words="english"
        )),
        ("clf", LinearSVC(
            C=best_params["C"],
            loss=best_params["loss"],
            class_weight="balanced",
            dual="auto",
            random_state=42
        ))
    ])

    # Train on full training set
    final_pipeline.fit(X_train, y_train)

    # Final Holdout Evaluation
    preds = final_pipeline.predict(X_val)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="macro")

    # Log Inputs & Metrics
    mlflow.log_params(best_params)
    mlflow.log_params({"text_source": "title_plus_description"}) # Explicitly log the source!
    mlflow.log_metrics({"val_accuracy": acc, "val_macro_f1": f1})

    # Save Model Artifact
    signature = infer_signature(X_val, preds)
    mlflow.sklearn.log_model(
        sk_model=final_pipeline,
        artifact_path="job_categorizer_model",
        signature=signature
    )

    print("\n✅ Champion Model Saved to DagsHub!")
    print(f"   Final Validation Accuracy: {acc:.4f}")
    print(f"   Final Validation Macro F1: {f1:.4f}")

In [ ]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

print(f"🧬 Preparing final freeze on {len(X_full)} total samples...")

final_production_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=best_params["max_features"],
        min_df=best_params["min_df"],
        ngram_range=best_params["ngram_range"],
        sublinear_tf=best_params["sublinear_tf"],
        stop_words="english"
    )),
    ("clf", LinearSVC(
        C=best_params["C"],
        loss=best_params["loss"],
        class_weight="balanced",
        dual=True,
        random_state=42
    ))
])


getting the params from best run


In [ ]:
import mlflow

# The run ID provided
RUN_ID = "8f0a412b5b5948118a307b38818ef430"

# Fetch the run data
run = mlflow.get_run(RUN_ID)

# Extract the parameters
params = run.data.params

# Display the results
print("--- Logged Parameters ---")
for key, value in params.items():
    print(f"{key}: {value}")

In [ ]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import ast  # For safe string-to-tuple conversion

# 1. Manually set your retrieved parameters
# These are the params you got from run 8f0a412b5b5948118a307b38818ef430
retrieved_params = {
    "max_features": 5000,
    "min_df": 2,
    "ngram_range": (1, 3), # Handled as tuple
    "sublinear_tf": False,
    "C": 0.22687699510834697,
    "loss": "squared_hinge"
}

# 2. Prepare Data
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])

print(f"🧬 Preparing final freeze on {len(X_full)} total samples...")

# 3. Define the Pipeline using the retrieved_params
final_production_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=int(retrieved_params["max_features"]),
        min_df=int(retrieved_params["min_df"]),
        ngram_range=retrieved_params["ngram_range"],
        sublinear_tf=bool(retrieved_params["sublinear_tf"]),
        stop_words="english"
    )),
    ("clf", LinearSVC(
        C=float(retrieved_params["C"]),
        loss=retrieved_params["loss"],
        class_weight="balanced",
        dual=True,
        random_state=42
    ))
])



PIPELINE BUILDING


In [ ]:
import mlflow.sklearn
from mlflow.models import infer_signature
import pandas as pd

# 1. FIX: Use .iloc[[:3]] to slice the first 3 rows as a DataFrame
# This preserves the column structure that MLflow requires.
input_example = X_full.iloc[:3].to_frame() if isinstance(X_full, pd.Series) else X_full.iloc[:3]

with mlflow.start_run(run_name="final_production_freeze_v5") as run:
    RUN_ID = run.info.run_id

    mlflow.log_params(retrieved_params)

    # Train
    final_production_pipeline.fit(X_full, y_full)

    # Create Signature
    signature = infer_signature(X_full, final_production_pipeline.predict(X_full))

    print(f"📦 Uploading artifacts to DagsHub for Run: {RUN_ID}...")

    # 2. Using 'name' (the new standard) instead of 'artifact_path'
    model_info = mlflow.sklearn.log_model(
        sk_model=final_production_pipeline,
        name="job_categorizer_model",
        signature=signature,
        input_example=input_example,
        registered_model_name="Job_Categorizer_Production"
    )

    print(f"✅ SUCCESS: Model registered and artifacts uploaded to DagsHub!")

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
# Register and set an alias called "champion" (the new 'Production')
client.set_registered_model_alias("Job_Categorizer_Production", "champion", 4)
print("✅ Alias '@champion' set for Version 1")

In [ ]:
import mlflow.sklearn

# Use the @alias syntax - this is the MLflow 3.0 standard
model_uri = "models:/Job_Categorizer_Production@champion"

print(f"📡 Fetching latest champion model from DagsHub...")
try:
    # This will pull the artifacts. If it hangs, check your DagsHub token!
    model = mlflow.sklearn.load_model(model_uri)
    print("💎 Model loaded successfully!")
except Exception as e:
    print(f"❌ Error: {e}. Check if Version 1 actually has files on DagsHub.")

In [ ]:
test_samples = [
    "Looking for a Machine Learning Engineer with experience in Python, sklearn, NLP, and model deployment.",
    "Frontend developer required with React, JavaScript, HTML, CSS, and UI optimization skills.",
    "Hiring a DevOps engineer to manage AWS infrastructure, CI/CD pipelines, Docker, and Kubernetes."
]

predictions = model.predict(test_samples)

for text, pred in zip(test_samples, predictions):
    print("📝 TEXT:", text)
    print("🎯 PREDICTED CATEGORY:", pred)
    print("-" * 80)
